# Data Prep

## Combinación de hojas de excel

In [1]:
import pandas as pd
import os

# Ruta al archivo Excel
file_path = "data/online_retail_II.xlsx"
combined_path = "data/online_retail.csv"

if os.path.exists(combined_path):
    print(f"El archivo {combined_path} ya existe. No se sobrescribirá.")
else:
    print("Leyendo datos, puede tardar unos minutos...")
    df1 = pd.read_excel(file_path, sheet_name='Year 2009-2010')
    df2 = pd.read_excel(file_path, sheet_name='Year 2010-2011')

    combined_df = pd.concat([df1, df2], ignore_index=True)
    combined_df.to_csv(combined_path, index=False)
    print(f"Datos combinados guardados en {combined_path}.")

El archivo data/online_retail.csv ya existe. No se sobrescribirá.


## Pre Procesamiento

### Carga y comprensión

In [2]:
# Lectura de datos
df = pd.read_csv(combined_path)

print(f"{df.shape[0]} filas y {df.shape[1]} columnas cargadas.")
print("-" * 10, "\n")
for col in df.columns:
    print(f"{col}: {df[col].dtype}")
print("-" * 10, "\n")
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df.head()

1067371 filas y 8 columnas cargadas.
---------- 

Invoice: object
StockCode: object
Description: object
Quantity: int64
InvoiceDate: object
Price: float64
Customer ID: float64
Country: object
---------- 

Primeras filas del DataFrame:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Conversión de tipos

In [3]:
# Conversion de columnas a tipo fecha
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

### Limpieza

In [4]:
# Conteo inicial de registros
original_rows = df.shape[0]

# Conteo de nulos por columnas clave
nulos_customer_id = df['Customer ID'].isna().sum()
nulos_description = df['Description'].isna().sum()
nulos_quantity = df['Quantity'].isna().sum()
nulos_price = df['Price'].isna().sum()

print("Registros con valores nulos:")
print(f"- Customer ID nulos: {nulos_customer_id}")
print(f"- Description nulos: {nulos_description}")
print(f"- Quantity nulos: {nulos_quantity}")
print(f"- Price nulos: {nulos_price}")

Registros con valores nulos:
- Customer ID nulos: 243007
- Description nulos: 4382
- Quantity nulos: 0
- Price nulos: 0


In [5]:
# Guardar registros antes de cada limpieza para comparar
rows_before = df.shape[0]

# Eliminar nulos
df = df.dropna(subset=['Customer ID'])
after_customer_id = df.shape[0]

df = df.dropna(subset=['Description'])
after_description = df.shape[0]

df = df.dropna(subset=['Quantity', 'Price'])
after_qty_price = df.shape[0]

# Eliminar valores no válidos
df = df[df['Quantity'] > 0]
after_qty = df.shape[0]

df = df[df['Price'] > 0]
after_price = df.shape[0]

# Reporte
print("Resumen de registros eliminados:")
print(f"- Original: {original_rows}")
print(f"- Después de eliminar nulos en Customer ID: {after_customer_id}  ({original_rows - after_customer_id} eliminados)")
print(f"- Después de eliminar nulos en Description: {after_description}  ({after_customer_id - after_description} eliminados)")
print(f"- Después de eliminar nulos en Quantity y Price: {after_qty_price}  ({after_description - after_qty_price} eliminados)")
print(f"- Después de eliminar Quantity <= 0: {after_qty}  ({after_qty_price - after_qty} eliminados)")
print(f"- Después de eliminar Price <= 0: {after_price}  ({after_qty - after_price} eliminados)")
print(f"→ Total final: {after_price} registros (eliminados: {original_rows - after_price})")


Resumen de registros eliminados:
- Original: 1067371
- Después de eliminar nulos en Customer ID: 824364  (243007 eliminados)
- Después de eliminar nulos en Description: 824364  (0 eliminados)
- Después de eliminar nulos en Quantity y Price: 824364  (0 eliminados)
- Después de eliminar Quantity <= 0: 805620  (18744 eliminados)
- Después de eliminar Price <= 0: 805549  (71 eliminados)
→ Total final: 805549 registros (eliminados: 261822)


### Creación de nuevas columnas

In [6]:
df["TotalSale"] = df["Quantity"] * df["Price"]
df["Devolucion"] = df["Invoice"].astype(str).str.startswith("C")
df["Mes"] = df["InvoiceDate"].dt.to_period("M").astype(str)

### Exportación

In [8]:
df.to_csv("data/retail_clean.csv", index=False)